# SleepEventNet 모델 구축 및 추론 파이프라인

이 노트북은 수면 중 발생 가능한 이벤트 (이갈이, 코골이, 기침, 잠꼬대, 정상 여부)를 분류하는 `SleepEventNet` 모델을 다음과 같은 순서로 구성하고 실행합니다

1. **TensorFlow 기반 모델 생성**
2. **TensorFlow 모델 추론 (test set)**
3. **TFLite 변환**
4. **TFLite 모델 추론**


In [1]:
from src.build_model import build_tensorflow_model,convert_tflite,load_tensorflow_model
from src.utils import predict_with_tf_model,predict_with_tflite_model

## Step 1: TensorFlow 모델 생성
YAMNet을 불러온 뒤 사용자 정의 분류 레이어(SleepEventClassificationLayer)를 연결하여 최종 모델을 생성.


In [2]:
build_tensorflow_model(yamnet_model_path= 'model/yamnet_tensorflow2_model',
                       save_path='model/SleepEventNet.h5')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ waveform (InputLayer)           │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ yamnet_wrapper (YamnetWrapper)  │ [(None, 521), (None,   │             0 │
│                                 │ 1024), (None, 64)]     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sleep_event_classification_lay… │ (1, 5)                 │             0 │
│ (SleepEventClassificationLayer) │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

<Functional name=functional, built=True>

## Step 2: TensorFlow 모델 추론
생성된 모델을 로드하여 수면 이벤트 샘플 데이터에 대해 예측을 수행.


입력은 20초 소리(wav)이며, 출력은 **[이갈이, 코골이, 기침, 수면 중 말소리, 정상 여부]에 대한 float 점수** 5개 리스트

In [4]:
model = load_tensorflow_model(tensorflow_model_path='model/SleepEventNet.h5')

predict_with_tf_model(tensorflow_model= model,
                      wav_file_path='data/20_seconds/bruxism/1/test1.wav')

predict_with_tf_model(tensorflow_model= model,
                      wav_file_path='data/20_seconds/cough/3/test2.wav')

predict_with_tf_model(tensorflow_model= model,
                      wav_file_path='data/20_seconds/snoring/3/test3.wav')

predict_with_tf_model(tensorflow_model= model,
                      wav_file_path='data/20_seconds/speech/1/test1.wav')

predict_with_tf_model(tensorflow_model= model,
                      wav_file_path='data/20_seconds/normal/segment_5.wav')

bruxism_score: 11.7600
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 9.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 7.9800
cough_score: 0.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 30.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 0.0000
is_normal: 1.0000



<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0., 0., 0., 0., 1.]], dtype=float32)>

## Step 3: TensorFlow 모델 → TFLite 변환
저장된 `.h5` 형식의 모델을 `.tflite` 포맷으로 변환하여 경량화.


In [5]:
convert_tflite(tensorflow_model_path= 'model/SleepEventNet.h5', 
               save_path= 'model/sleep_evnt_model_tflite.tflite')

INFO:tensorflow:Assets written to: C:\Users\myhom\AppData\Local\Temp\tmphruk7b5x\assets


INFO:tensorflow:Assets written to: C:\Users\myhom\AppData\Local\Temp\tmphruk7b5x\assets


Saved artifact at 'C:\Users\myhom\AppData\Local\Temp\tmphruk7b5x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None), dtype=tf.float32, name='waveform')
Output Type:
  TensorSpec(shape=(1, 5), dtype=tf.float32, name=None)
Captures:
  2350424118608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424116880: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  2350424118416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424118224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424118032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424117648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424115920: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  2350424117456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424117264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424117072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2350424116688: 

## Step 4: TFLite 모델 추론
변환된 TFLite 모델을 이용하여, 이전 TensorFlow 모델과 동일한 입력 데이터에 대해 추론을 수행.

입력은 20초 소리(wav)이며, 출력은 **[이갈이, 코골이, 기침, 수면 중 말소리, 정상 여부]에 대한 float 점수** 5개 리스트

In [2]:
predict_with_tflite_model(tflite_model_path='model/sleep_evnt_model_tflite.tflite',
                      wav_file_path='data/20_seconds/bruxism/1/test1.wav')

predict_with_tflite_model(tflite_model_path='model/sleep_evnt_model_tflite.tflite',
                      wav_file_path='data/20_seconds/cough/3/test2.wav')

predict_with_tflite_model(tflite_model_path='model/sleep_evnt_model_tflite.tflite',
                      wav_file_path='data/20_seconds/snoring/3/test3.wav')

predict_with_tflite_model(tflite_model_path='model/sleep_evnt_model_tflite.tflite',
                      wav_file_path='data/20_seconds/speech/1/test1.wav')

predict_with_tflite_model(tflite_model_path='model/sleep_evnt_model_tflite.tflite',
                      wav_file_path='data/20_seconds/normal/segment_5.wav')


bruxism_score: 11.7600
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 9.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 7.9800
cough_score: 0.0000
speech_score: 0.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 30.0000
is_normal: 0.0000

bruxism_score: 0.0000
snoring_score: 0.0000
cough_score: 0.0000
speech_score: 0.0000
is_normal: 1.0000

